# Partie III - Application du machine learning pour la fonction d'évaluation

## Deuxième modèle : Arbre de décision

In [14]:
import pandas as pd
import numpy as np
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Charger les fichiers CSV
gagnants = pd.read_csv("dataset_fanorona/fanorona_winning_positions_cleaned.csv", header=None)  # Remplace avec le bon chemin
perdants = pd.read_csv("dataset_fanorona/fanorona_losing_positions_cleaned.csv", header=None)

# Ajouter la colonne cible (1 pour gagnant, -1 pour perdant)
gagnants["score"] = 1
perdants["score"] = -1

# Fusionner les données
data = pd.concat([gagnants, perdants], ignore_index=True)

# Séparer les features et labels
X = data.iloc[:, :-1].values  # Les configurations de plateau
y = data["score"].values  # Les scores

# Diviser en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner l'arbre de décision
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Évaluer le modèle
y_pred = clf.predict(X_test)
print("Précision du modèle :", accuracy_score(y_test, y_pred))

# Classe du jeu Fanorona Telo
class FanoronaTelo:
    def __init__(self):
        self.board = [0] * 9  # 0 = vide, 1 = blanc, -1 = noir
        self.current_player = 1
        self.placed_pieces = {1: 0, -1: 0}  # Nombre de pièces placées par joueur
        self.phase = "placement"  # Phase du jeu : "placement" ou "déplacement"
    
    def get_valid_moves(self):
        if self.phase == "placement":
            return [i for i in range(9) if self.board[i] == 0]
        else:
            moves = []
            for i in range(9):
                if self.board[i] == self.current_player:
                    voisins = self.get_neighbors(i)
                    for v in voisins:
                        if self.board[v] == 0:
                            moves.append((i, v))  # (position actuelle, nouvelle position)
            return moves
    
    def get_neighbors(self, pos):
        neighbors = {
            0: [1, 3, 4], 1: [0, 2, 4], 2: [1, 4, 5],
            3: [0, 4, 6], 4: [0, 1, 2, 3, 5, 6, 7, 8], 5: [2, 4, 8],
            6: [3, 4, 7], 7: [4, 6, 8], 8: [4, 5, 7]
        }
        return neighbors[pos]
    
    def make_move(self, move):
        if self.phase == "placement":
            if self.board[move] == 0:
                self.board[move] = self.current_player
                self.placed_pieces[self.current_player] += 1
                if self.placed_pieces[1] == 3 and self.placed_pieces[-1] == 3:
                    self.phase = "déplacement"
                self.current_player *= -1
                return True
        else:
            src, dst = move
            if self.board[src] == self.current_player and self.board[dst] == 0:
                self.board[src] = 0
                self.board[dst] = self.current_player
                self.current_player *= -1
                return True
        return False
    
    def is_winning(self, player):
        winning_positions = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]
        return any(all(self.board[i] == player for i in pos) for pos in winning_positions)
    
    def is_draw(self):
        return len(self.get_valid_moves()) == 0
    
    def get_state(self):
        return self.board.copy()

# Fonction IA pour choisir le meilleur coup
def choose_best_move(game):
    best_move = None
    best_score = -float('inf') if game.current_player == 1 else float('inf')
    
    for move in game.get_valid_moves():
        temp_board = game.get_state()
        if game.phase == "placement":
            temp_board[move] = game.current_player
        else:
            src, dst = move
            temp_board[src] = 0
            temp_board[dst] = game.current_player
        
        score = clf.predict([temp_board])[0]
        
        if (game.current_player == 1 and score > best_score) or (game.current_player == -1 and score < best_score):
            best_score = score
            best_move = move
    
    return best_move

# Simulation du jeu
print("Début du jeu Fanorona Telo avec IA")
game = FanoronaTelo()
while True:
    print("Plateau actuel:", game.board)
    if game.current_player == 1:
        move = choose_best_move(game)
        print(f"L'IA Blanc joue en position {move}")
    else:
        move = choose_best_move(game)  # IA Noir joue aussi stratégiquement
        print(f"L'IA Noir joue en position {move}")
    
    game.make_move(move)
    
    if game.is_winning(1):
        print("Les Blancs gagnent!")
        break
    elif game.is_winning(-1):
        print("Les Blancs perdent!")
        break
    elif game.is_draw():
        print("Match nul!")
        break

Précision du modèle : 0.9180327868852459
Début du jeu Fanorona Telo avec IA
Plateau actuel: [0, 0, 0, 0, 0, 0, 0, 0, 0]
L'IA Blanc joue en position 0
Plateau actuel: [1, 0, 0, 0, 0, 0, 0, 0, 0]
L'IA Noir joue en position 1
Plateau actuel: [1, -1, 0, 0, 0, 0, 0, 0, 0]
L'IA Blanc joue en position 2
Plateau actuel: [1, -1, 1, 0, 0, 0, 0, 0, 0]
L'IA Noir joue en position 4
Plateau actuel: [1, -1, 1, 0, -1, 0, 0, 0, 0]
L'IA Blanc joue en position 3
Plateau actuel: [1, -1, 1, 1, -1, 0, 0, 0, 0]
L'IA Noir joue en position 5
Plateau actuel: [1, -1, 1, 1, -1, -1, 0, 0, 0]
L'IA Blanc joue en position (3, 6)
Plateau actuel: [1, -1, 1, 0, -1, -1, 1, 0, 0]
L'IA Noir joue en position (4, 3)
Plateau actuel: [1, -1, 1, -1, 0, -1, 1, 0, 0]
L'IA Blanc joue en position (0, 4)
Les Blancs gagnent!
